# Detecção de Fraude em contratos de seguros: Treine, Verifique Viés, Sintonize, Registre Linhagem e Registre um Modelo

## Background

Este notebook é a segunda parte de uma série de notebooks que demonstrarão como preparar, treinar e implantar um modelo que detecta reclamações automáticas fraudulentas. Neste notebook, estaremos preparando, processando e armazenando recursos usando o SageMaker Feature Store. Você pode optar por executar este notebook sozinho ou em sequência com os outros notebooks listados abaixo. Consulte o [README.md](README.md) para obter mais informações sobre este caso de uso implementado por esta série de notebooks.

1. **[Detecção de Fraude em contratos de seguros: exploração de dados](./0-AutoClaimFraudDetection.ipynb)**
1. [Detecção de Fraude em contratos de seguros: preparação de dados, processo e recursos de armazenamento](./1-data-prep-e2e.ipynb)
1. [Detecção de Fraude em contratos de seguros: treinar, verificar viés, ajustar, registrar linhagem e registrar um modelo](./2-lineage-train-assess-bias-tune-registry-e2e.ipynb)
1. [Detecção de Fraude em contratos de seguros: mitigar, treinar, registrar e implantar modelo imparcial](./3-mitigate-bias-train-model2-registry-e2e.ipynb)


## Conteúdo
1. [Arquitetura para preparação de dados, recursos e armazenamento](#Arquitetura para preparação de dados, recursos de processo e armazenamento)
1. [Introdução: Criando recursos](#Getting-Started:-Creating-Resources)
1. [Conjuntos de dados e tipos de recursos](#Datasets-and-Feature-Types)
1. [SageMaker Feature Store](#SageMaker-Feature-Store)
1. [Criar conjuntos de dados de treinamento e teste](#Create-Train-and-Test-Datasets)

## Arquitetura para o estágio do ciclo de vida de ML: treinar, verificar viés, ajustar, gravar linhagem, registrar modelo
----

![train-assess-tune-register](./images/e2e-2-pipeline-v3b.png)

### Instale as bibliotecas necessárias e/ou atualize

In [2]:
!python -m pip install -Uq pip
!python -m pip install -q awswrangler==2.2.0 imbalanced-learn==0.7.0 sagemaker==2.41.0 boto3==1.17.70

### Importar bibliotecas

In [3]:
import json
import time
import boto3
import sagemaker
import numpy as np
import pandas as pd
import awswrangler as wr

from sagemaker.xgboost.estimator import XGBoost
from model_package_src.inference_specification import InferenceSpecification

### Selecione a região, boto3 e SageMaker SDK 

In [4]:
# You can change this to a region of your choice
import sagemaker

region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

Using AWS Region: us-east-1


In [5]:
boto3.setup_default_session(region_name=region)

boto_session = boto3.Session(region_name=region)

s3_client = boto3.client("s3", region_name=region)

sagemaker_boto_client = boto_session.client("sagemaker")

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_boto_client
)

sagemaker_role = sagemaker.get_execution_role()

account_id = boto3.client("sts").get_caller_identity()["Account"]

In [6]:
# variables used for parameterizing the notebook run
bucket = sagemaker_session.default_bucket()
prefix = "fraud-detect-demo"

estimator_output_path = f"s3://{bucket}/{prefix}/training_jobs"
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"

bias_report_1_output_path = f"s3://{bucket}/{prefix}/clarify-output/bias_1"


xgb_model_name = "xgb-insurance-claims-fraud-model"
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"
predictor_instance_count = 1
predictor_instance_type = "ml.c5.xlarge"
batch_transform_instance_count = 1
batch_transform_instance_type = "ml.c5.xlarge"
claify_instance_count = 1
clairfy_instance_type = "ml.c5.xlarge"

### Armazenamento de dados

In [7]:
train_data_uri = f"s3://{bucket}/{prefix}/data/train/train.csv"
test_data_uri = f"s3://{bucket}/{prefix}/data/test/test.csv"


s3_client.upload_file(
    Filename="data/train.csv", Bucket=bucket, Key=f"{prefix}/data/train/train.csv"
)
s3_client.upload_file(Filename="data/test.csv", Bucket=bucket, Key=f"{prefix}/data/test/test.csv")

## Treine um modelo usando o XGBoost
----

Depois que os conjuntos de dados de treinamento e teste persistirem no S3, você poderá começar a treinar um modelo definindo qual SageMaker Estimator você gostaria de usar. Para este guia, você usará o [XGBoost Open Source Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/xgboost.html) para treinar seu modelo. Esse estimador é acessado por meio do SDK do SageMaker, mas espelha a versão de código aberto do [pacote XGBoost Python](https://xgboost.readthedocs.io/en/latest/python/index.html). Qualquer funcionalidade fornecida pelo pacote XGBoost Python pode ser implementada em seu script de treinamento.

### Defina os hiperparâmetros
Estes são os parâmetros que serão enviados ao nosso script de treinamento para treinar o modelo. Embora todos sejam definidos como "hiperparâmetros" aqui, eles podem abranger os [Parâmetros da tarefa de aprendizado](https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters) do XGBoost, [Parâmetros do impulsionador da árvore ](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster) ou qualquer outro parâmetro que você queira configurar para o XGBoost.

In [8]:
hyperparameters = {
    "max_depth": "3",
    "eta": "0.2",
    "objective": "binary:logistic",
    "num_round": "100",
}

### Crie e ajuste o estimador
Se você quiser explorar a amplitude de funcionalidades oferecidas pelo SageMaker XGBoost Framework, você pode ler sobre todos os parâmetros de configuração referenciando as classes herdadas. A classe XGBoost herda da classe Framework e o Framework herda da classe EstimatorBase:
* [Documentação do XGBoost Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/xgboost.html#sagemaker.xgboost.estimator.XGBoost)
* [Documentação da estrutura](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Framework)
* [Documentação do EstimatorBase](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)

In [9]:
xgb_estimator = XGBoost(
    entry_point="xgboost_starter_script.py",
    output_path=estimator_output_path,
    code_location=estimator_output_path,
    hyperparameters=hyperparameters,
    role=sagemaker_role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    framework_version="1.0-1",
)

In [10]:
if "training_job_1_name" not in locals():

    xgb_estimator.fit(inputs={"train": train_data_uri})
    training_job_1_name = xgb_estimator.latest_training_job.job_name

else:
    print(f"Using previous training job: {training_job_1_name}")

2022-08-28 11:23:02 Starting - Starting the training job...
2022-08-28 11:23:27 Starting - Preparing the instances for trainingProfilerReport-1661685782: InProgress
.........
2022-08-28 11:24:54 Downloading - Downloading input data...
2022-08-28 11:25:16 Training - Downloading the training image......
2022-08-28 11:26:32 Uploading - Uploading generated training modelINFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Created wheel for xgboost-starter-script: filename=xgboost_st

## Linhagem, Artefatos e Associações
----

O Amazon SageMaker ML Lineage Tracking cria e armazena informações sobre as etapas de um fluxo de trabalho de machine learning (ML), desde a preparação de dados até a implantação do modelo. Com as informações de rastreamento, você pode reproduzir as etapas do fluxo de trabalho, rastrear a linhagem do modelo e do conjunto de dados e estabelecer padrões de governança e auditoria do modelo. Com o SageMaker Lineage Tracking, os cientistas de dados e construtores de modelos podem fazer o seguinte:
* Mantenha um histórico de execução de experimentos de descoberta de modelo.
* Estabelecer governança de modelo rastreando artefatos de linhagem de modelo para auditoria e verificação de conformidade.
* Clone e execute novamente fluxos de trabalho para experimentar cenários hipotéticos durante o desenvolvimento de modelos.
* Compartilhe um fluxo de trabalho que os colegas possam reproduzir e aprimorar (por exemplo, enquanto colaboram na solução de um problema de negócios).
* Clone e execute novamente fluxos de trabalho com rotinas adicionais de depuração ou registro, ou novas variações de entrada para solucionar problemas em modelos de produção.

### Registrar artefatos

Embora o objeto `xgb_estimator` retenha muitos dados que precisamos aprender sobre como o modelo foi treinado, ele é, de fato, um objeto efêmero que o SageMaker não persiste e não pode ser reinstanciado posteriormente. Embora percamos algumas de suas conveniências quando ele se for, ainda podemos recuperar todos os dados de que precisamos acessando os trabalhos de treinamento que ele criou.

In [11]:
training_job_1_info = sagemaker_boto_client.describe_training_job(
    TrainingJobName=training_job_1_name
)

#### Artefato de código

In [12]:
# return any existing artifact which match the our training job's code arn
# ====>

# extract the training code uri and check if it's an exisiting artifact
code_s3_uri = training_job_1_info["HyperParameters"]["sagemaker_submit_directory"]

matching_artifacts = list(
    sagemaker.lineage.artifact.Artifact.list(
        source_uri=code_s3_uri, sagemaker_session=sagemaker_session
    )
)

# use existing arifact if it's already been created, otherwise create a new artifact
if matching_artifacts:
    code_artifact = matching_artifacts[0]
    print(f"Using existing artifact: {code_artifact.artifact_arn}")
else:
    code_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name="TrainingScript",
        source_uri=code_s3_uri,
        artifact_type="Code",
        sagemaker_session=sagemaker_session,
    )
    print(f"Create artifact {code_artifact.artifact_arn}: SUCCESSFUL")

Create artifact arn:aws:sagemaker:us-east-1:173965381620:artifact/30fe27721dedf7def783cf66bbfcd08b: SUCCESSFUL


#### Artefato de dados de treinamento

In [13]:
training_data_s3_uri = training_job_1_info["InputDataConfig"][0]["DataSource"]["S3DataSource"][
    "S3Uri"
]

matching_artifacts = list(
    sagemaker.lineage.artifact.Artifact.list(
        source_uri=training_data_s3_uri, sagemaker_session=sagemaker_session
    )
)

if matching_artifacts:
    training_data_artifact = matching_artifacts[0]
    print(f"Using existing artifact: {training_data_artifact.artifact_arn}")
else:
    training_data_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name="TrainingData",
        source_uri=training_data_s3_uri,
        artifact_type="Dataset",
        sagemaker_session=sagemaker_session,
    )
    print(f"Create artifact {training_data_artifact.artifact_arn}: SUCCESSFUL")

Using existing artifact: arn:aws:sagemaker:us-east-1:173965381620:artifact/661194bded7858c1b18cf2bb30b5bce8


#### Artefato de modelo

In [14]:
trained_model_s3_uri = training_job_1_info["ModelArtifacts"]["S3ModelArtifacts"]

matching_artifacts = list(
    sagemaker.lineage.artifact.Artifact.list(
        source_uri=trained_model_s3_uri, sagemaker_session=sagemaker_session
    )
)

if matching_artifacts:
    model_artifact = matching_artifacts[0]
    print(f"Using existing artifact: {model_artifact.artifact_arn}")
else:
    model_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name="TrainedModel",
        source_uri=trained_model_s3_uri,
        artifact_type="Model",
        sagemaker_session=sagemaker_session,
    )
    print(f"Create artifact {model_artifact.artifact_arn}: SUCCESSFUL")

Using existing artifact: arn:aws:sagemaker:us-east-1:173965381620:artifact/5e38d21d7eed4c9128ce796f9d9c2223


### Definir associações de artefatos

In [15]:
trial_component = sagemaker_boto_client.describe_trial_component(
    TrialComponentName=training_job_1_name + "-aws-training-job"
)
trial_component_arn = trial_component["TrialComponentArn"]

#### Artefatos de entrada

In [16]:
input_artifacts = [code_artifact, training_data_artifact]

for a in input_artifacts:
    try:
        sagemaker.lineage.association.Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type="ContributedTo",
            sagemaker_session=sagemaker_session,
        )
        print(f"Association with {a.artifact_type}: SUCCEESFUL")
    except:
        print(f"Association already exists with {a.artifact_type}")

Association with Code: SUCCEESFUL
Association already exists with DataSet


#### Saída de Artefatos

In [17]:
output_artifacts = [model_artifact]

for a in output_artifacts:
    try:
        sagemaker.lineage.association.Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type="Produced",
            sagemaker_session=sagemaker_session,
        )
        print(f"Association with {a.artifact_type}: SUCCESSFUL")
    except:
        print(f"Association already exists with {a.artifact_type}")

Association with Model: SUCCESSFUL


## Avalie o modelo de viés com o Clarify
----

O Amazon SageMaker Clarify ajuda a melhorar seus modelos de machine learning (ML) detectando possíveis vieses e ajudando a explicar as previsões que os modelos fazem. Ele ajuda a identificar vários tipos de viés nos dados de pré-treinamento e pós-treinamento que podem surgir durante o treinamento do modelo ou quando o modelo está em produção. O SageMaker Clarify ajuda a explicar como esses modelos fazem previsões usando uma abordagem de atribuição de recursos. Ele também monitora as inferências feitas pelos modelos em produção para desvios ou desvios de atribuição de recursos. A funcionalidade de imparcialidade e explicabilidade fornecida pelo SageMaker Clarify fornece componentes que ajudam os clientes da AWS a criar modelos de machine learning menos tendenciosos e mais compreensíveis. Ele também fornece ferramentas para ajudá-lo a gerar relatórios de governança de modelo que você pode usar para informar as equipes de risco e conformidade e reguladores externos.

Você pode consultar o [Guia do desenvolvedor do SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-fairness-and-explainability.html) para obter mais informações sobre o SageMaker Clarify.

### Criar modelo a partir do estimador

In [18]:
model_1_name = f"{prefix}-xgboost-pre-smote"
model_matches = sagemaker_boto_client.list_models(NameContains=model_1_name)["Models"]

if not model_matches:

    model_1 = sagemaker_session.create_model_from_job(
        name=model_1_name,
        training_job_name=training_job_1_info["TrainingJobName"],
        role=sagemaker_role,
        image_uri=training_job_1_info["AlgorithmSpecification"]["TrainingImage"],
    )
else:

    print(f"Model {model_1_name} already exists.")

### Verifique o viés do conjunto de dados e o viés do modelo

Com o SageMaker, podemos verificar o viés pré e pós-treinamento. As métricas de pré-treinamento mostram viés preexistente nesses dados, enquanto as métricas de pós-treinamento mostram viés nas previsões do modelo. Usando o SDK do SageMaker, podemos especificar em quais grupos queremos verificar o viés e quais métricas gostaríamos de mostrar.

Para executar o trabalho completo do Clarify, você deve descomentar o código na célula abaixo. A execução do trabalho levará cerca de 15 minutos. Se você deseja economizar tempo, pode visualizar os resultados na próxima célula, após a qual carrega uma saída pré-gerada se nenhum trabalho de polarização foi executado.

In [19]:
train_cols = wr.s3.read_csv(training_data_s3_uri).columns.to_list()

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=sagemaker_role,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    sagemaker_session=sagemaker_session,
)

bias_data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_uri,
    s3_output_path=bias_report_1_output_path,
    label="fraud",
    headers=train_cols,
    dataset_type="text/csv",
)

model_config = sagemaker.clarify.ModelConfig(
    model_name=model_1_name,
    instance_type=train_instance_type,
    instance_count=1,
    accept_type="text/csv",
)

predictions_config = sagemaker.clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

bias_config = sagemaker.clarify.BiasConfig(
    label_values_or_threshold=[0],
    facet_name="customer_gender_female",
    facet_values_or_threshold=[1],
)

# un-comment the code below to run the whole job

# if 'clarify_bias_job_1_name' not in locals():

#     clarify_processor.run_bias(
#         data_config=bias_data_config,
#         bias_config=bias_config,
#         model_config=model_config,
#         model_predicted_label_config=predictions_config,
#         pre_training_methods='all',
#         post_training_methods='all')

#     clarify_bias_job_1_name = clarify_processor.latest_job.name
#     %store clarify_bias_job_1_name

# else:
#     print(f'Clarify job {clarify_bias_job_name} has already run successfully.')

Os resultados serão armazenados em `/opt/ml/processing/output/report.pdf`
O treinamento para atingir mais de 90 por cento de precisão de classificação pode ser facilmente possível em um problema de classificação desequilibrada.

Assim, as expectativas desenvolvidas em relação à precisão da classificação que são na realidade contingentes a distribuições de classes equilibradas levarão a suposições e conclusões erradas e enganosas: induzir o cientista de dados e os espectadores a acreditar que um modelo tem desempenho extremamente alto quando, na verdade, não tem.

### Ver resultados do trabalho de esclarecimento (atalho)
A execução do Clarify em seu conjunto de dados ou modelo pode levar cerca de 15 minutos. Se você não tiver tempo para executar o trabalho, poderá visualizar os resultados pré-gerados incluídos nesta demonstração. Caso contrário, você pode executar o trabalho removendo o comentário do código na célula acima.

In [20]:
if "clarify_bias_job_1_name" in locals():
    s3_client.download_file(
        Bucket=bucket,
        Key=f"{prefix}/clarify-output/bias_1/analysis.json",
        Filename="clarify_output/bias_1/analysis.json",
    )
    print(f"Downloaded analysis from previous Clarify job: {clarify_bias_job_1_name}")
else:
    print(f"Loading pre-generated analysis file...")

with open("clarify_output/bias_1/analysis.json", "r") as f:
    bias_analysis = json.load(f)

results = bias_analysis["pre_training_bias_metrics"]["facets"]["customer_gender_female"][0][
    "metrics"
][1]
print(json.dumps(results, indent=4))

Loading pre-generated analysis file...
{
    "name": "CI",
    "description": "Class Imbalance (CI)",
    "value": 0.398
}


Neste conjunto de dados de exemplo, os dados são tendenciosos em relação ao sexo feminino, com apenas 38,9% das amostras de dados de clientes do sexo feminino. Abordaremos isso no próximo caderno, onde mostramos como mitigamos esse viés de desequilíbrio de classe. Embora estejamos abordando apenas o Desequilíbrio de Classe como um exemplo de estatísticas de viés, você também pode levar em consideração muitos outros fatores de viés. Para obter mais detalhes, consulte: [Fairness Measures for Machine Learning in Finance](https://pages.awscloud.com/rs/112-TZM-766/images/Fairness.Measures.for.Machine.Learning.in.Finance. pdf)

para um exemplo mais detalhado veja [este](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb) exemplo do github.

For more detailed resulst let's look at the generated report, that can be found here: `s3://{bucket}/e2e-fraud-detect/clarify/bias-2/report.pdf`

In [21]:
# uncomment to copy report and view
#!aws s3 cp s3://{bucket}/{prefix}/clarify-output/bias_1/report.pdf ./clarify_output

## Modelo de Depósito e Linhagem no Registro de Modelos do SageMaker
----

Uma vez que um modelo útil tenha sido treinado e seus artefatos devidamente associados, a próxima etapa é salvar o modelo em um registro para referência futura e possível implantação.

### Criar grupo de pacotes de modelo
Um Grupos de Pacotes de Modelos contém várias versões ou iterações de um modelo. Embora não seja necessário criá-los para cada modelo no registro, eles ajudam a organizar vários modelos que têm a mesma finalidade e fornecem controle de versão automático.

In [22]:
if "mpg_name" not in locals():
    mpg_name = prefix
    print(f"Model Package Group name: {mpg_name}")

Model Package Group name: fraud-detect-demo


In [23]:
mpg_input_dict = {
    "ModelPackageGroupName": mpg_name,
    "ModelPackageGroupDescription": "Insurance claim fraud detection",
}

In [24]:
matching_mpg = sagemaker_boto_client.list_model_package_groups(NameContains=mpg_name)['ModelPackageGroupSummaryList']

if matching_mpg:
    print(f'Using existing Model Package Group: {mpg_name}')
else:
    mpg_response = sagemaker_boto_client.create_model_package_group(**mpg_input_dict)
    print(f'Create Model Package Group {mpg_name}: SUCCESSFUL')
    %store mpg_name

Create Model Package Group fraud-detect-demo: SUCCESSFUL
Stored 'mpg_name' (str)


### Criar pacote de modelo para modelo treinado

#### Crie e carregue um relatório de métricas

In [25]:
model_metrics_report = {"binary_classification_metrics": {}}
for metric in training_job_1_info["FinalMetricDataList"]:
    stat = {metric["MetricName"]: {"value": metric["Value"], "standard_deviation": "NaN"}}
    model_metrics_report["binary_classification_metrics"].update(stat)

with open("training_metrics.json", "w") as f:
    json.dump(model_metrics_report, f)

metrics_s3_key = (
    f"{prefix}/training_jobs/{training_job_1_info['TrainingJobName']}/training_metrics.json"
)
s3_client.upload_file(Filename="training_metrics.json", Bucket=bucket, Key=metrics_s3_key)

#### Defina a especificação de inferência

In [26]:
mp_inference_spec = InferenceSpecification().get_inference_specification_dict(
    ecr_image=training_job_1_info["AlgorithmSpecification"]["TrainingImage"],
    supports_gpu=False,
    supported_content_types=["text/csv"],
    supported_mime_types=["text/csv"],
)

mp_inference_spec["InferenceSpecification"]["Containers"][0]["ModelDataUrl"] = training_job_1_info[
    "ModelArtifacts"
]["S3ModelArtifacts"]

#### Definir métricas do modelo
Métricas diferentes da qualidade e do viés do modelo podem ser definidas. Consulte a documentação do Boto3 para [criar um pacote de modelo](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model_package).

In [27]:
model_metrics = {
    "ModelQuality": {
        "Statistics": {
            "ContentType": "application/json",
            "S3Uri": f"s3://{bucket}/{metrics_s3_key}",
        }
    },
    "Bias": {
        "Report": {
            "ContentType": "application/json",
            "S3Uri": f"{bias_report_1_output_path}/analysis.json",
        }
    },
}

In [28]:
mp_input_dict = {
    "ModelPackageGroupName": mpg_name,
    "ModelPackageDescription": "XGBoost classifier to detect insurance fraud.",
    "ModelApprovalStatus": "PendingManualApproval",
    "ModelMetrics": model_metrics,
}

mp_input_dict.update(mp_inference_spec)
mp1_response = sagemaker_boto_client.create_model_package(**mp_input_dict)

### Aguarde até que o pacote do modelo seja concluído

In [29]:
mp_info = sagemaker_boto_client.describe_model_package(
    ModelPackageName=mp1_response["ModelPackageArn"]
)
mp_status = mp_info["ModelPackageStatus"]

while mp_status not in ["Completed", "Failed"]:
    time.sleep(5)
    mp_info = sagemaker_boto_client.describe_model_package(
        ModelPackageName=mp1_response["ModelPackageArn"]
    )
    mp_status = mp_info["ModelPackageStatus"]
    print(f"model package status: {mp_status}")
print(f"model package status: {mp_status}")

model package status: Completed


### Ver pacote de modelo no registro

In [30]:
sagemaker_boto_client.list_model_packages(ModelPackageGroupName=mpg_name)["ModelPackageSummaryList"]

[{'ModelPackageGroupName': 'fraud-detect-demo',
  'ModelPackageVersion': 1,
  'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:173965381620:model-package/fraud-detect-demo/1',
  'ModelPackageDescription': 'XGBoost classifier to detect insurance fraud.',
  'CreationTime': datetime.datetime(2022, 8, 28, 11, 27, 21, 592000, tzinfo=tzlocal()),
  'ModelPackageStatus': 'Completed',
  'ModelApprovalStatus': 'PendingManualApproval'}]